# CP-SAT: Non-linear functions

In [1]:
from ortools.sat.python import cp_model

import numpy as np
from typing import Dict
from math import exp, log

FLOAT_APPROX_PRECISION = 100

In [2]:
from utils import create_boolean_is_equal_to

## CP-SAT doesn't want you to use non-linear functions

CP SAT supports many integer operations. But you can't compose an IntVar by a custom function, in particular if it's non-linear. 

For example, you can't do exp(var).

In [3]:
model = cp_model.CpModel()

x = model.NewIntVar(0, 100, "x")
result = model.NewIntVar(0, 100 * 100, "result")

# Let's say we want result == exp(x)
# This will fail (rightfully)
model.Add(result == exp(x))

TypeError: must be real number, not IntVar

## Precomputing the function values

Since integer variables take a finite amount of values, we can simply pre-compute the non-linear function values, and store them in a map.

Then, we set the image value to be equal to the value in the mapping when needed.

In [4]:
def lookup_value_in_dict(
    model: cp_model.CpModel,
    key_var: cp_model.IntVar,
    mapping: Dict[int, int],
    mapping_name: str = "mapping",
):
    """Creates a new variable equals to mapping[key_var]
    If the value is absent from the mapping, the value_var takes the value 0.
    """
    value_var = model.NewIntVar(
        min(mapping.values()), max(mapping.values()), f"{mapping_name}_{key_var.Name()}"
    )
    for mapping_key, mapping_value in mapping.items():
        key_var_is_equal_to = create_boolean_is_equal_to(model, key_var, mapping_key)
        model.Add(value_var == mapping_value).OnlyEnforceIf(key_var_is_equal_to)

    return value_var

Example with the exponential and natural logarithm function

In [412]:
# TODO : Make this work with variable bounds

def exp_of_x(
    model: cp_model.CpModel,
    var: cp_model.IntVar,
    float_precision_var=FLOAT_APPROX_PRECISION,
    float_precision_image=FLOAT_APPROX_PRECISION,
):
    lb = var.Proto().domain[0]
    ub = var.Proto().domain[1]
    x_to_exp_x = {
        x: round(exp(x / float_precision_var) * float_precision_image)
        for x in range(lb, ub + 1)
    }
    exp_of_var = lookup_value_in_dict(model, var, x_to_exp_x, mapping_name="exp")
    return exp_of_var

def log_of_x(
    model: cp_model.CpModel,
    var: cp_model.IntVar,
    float_precision_var=FLOAT_APPROX_PRECISION,
    float_precision_image=FLOAT_APPROX_PRECISION,
):
    # Log is only defined for x > 0
    lb = max(var.Proto().domain[0], 1)
    ub = max(var.Proto().domain[1], 1)
    assert lb <= ub
    x_to_log_x = {
        x: round(log(x / float_precision_var) * float_precision_image)
        for x in range(lb, ub + 1)
    }
    log_of_var = lookup_value_in_dict(model, var, x_to_log_x, mapping_name="log")
    return log_of_var

Here's how it would be used in a model.

In [413]:
float_precision_var = 100
float_precision_image = 10_000
value_of_x = 0.69

model = cp_model.CpModel()

x = model.NewIntVar(-10 * float_precision_var, 10 * float_precision_var, "x")
exp_x_var = exp_of_x(
    model,
    x,
    float_precision_var=float_precision_var,
    float_precision_image=float_precision_image,
)
log_x_var = log_of_x(
    model,
    x,
    float_precision_var=float_precision_var,
    float_precision_image=float_precision_image,
)
model.Add(x == round(float_precision_var * value_of_x))

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")

print(
    f"Solver value of exp({value_of_x}) is: {solver.Value(exp_x_var)/ float_precision_image} "
    + f"(Real value: {exp(value_of_x)})"
)
print(
    f"Solver value of log({value_of_x}) is: {solver.Value(log_x_var)/ float_precision_image} "
    + f"(Real value: {log(value_of_x)})"
)


Status = OPTIMAL
Solver value of exp(0.69) is: 1.9937 (Real value: 1.9937155332430823)
Solver value of log(0.69) is: -0.3711 (Real value: -0.37106368139083207)


## Application: sum of logarithms

### Approximate product of large numbers

Multiplying large int values can become difficult as we reach the upper limit fast. For example, we can't store the product of x and y below, because the result _could_ be higher than `2**63 - 1`, which is the upper bound for ints in CP-SAT. 

In [414]:
model = cp_model.CpModel()

x = model.NewIntVar(10**12 - 1000, 10**12 + 1000, "x")
y = model.NewIntVar(10**12 - 1000, 10**12 + 1000, "y")

# This will fail because the upper bound is above 2**63 - 1
x_times_y = model.NewIntVar(1, 10**10 * 10**10, "y")

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. ortools.util.python.sorted_interval_list.Domain(arg0: int, arg1: int)

Invoked with: 1, 100000000000000000000

Using a lower upper bound for the variable equal to the multiplication of the two others will fail. 

In [415]:
model = cp_model.CpModel()

x = model.NewIntVar(10**12 - 1000, 10**12 + 1000, "x")
y = model.NewIntVar(10**12 - 1000, 10**12 + 1000, "y")

# If we set the 
x_times_y = model.NewIntVar(1, 2**63 - 1, "y")
model.AddMultiplicationEquality(x_times_y, [x, y])

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")

Status = MODEL_INVALID


A trick is to use the logarithm property : `log(A) + log(B) = log(A*B)`. This allows us to deal with much smaller number. Indeed, `log(2**63-1) ~= 43.66`.

However, since we are dealing with decimal approximations, we must add a lot of decimals to keep estimations precise enough. 

In [416]:
model = cp_model.CpModel()

x = model.NewIntVar(10**12 - 1000, 10**12 + 1000, "x")
y = model.NewIntVar(10**12 - 1000, 10**12 + 1000, "y")

big_x_value = 10**12 + 10
big_y_value = 10**12 - 3
log_float_precision = 100_000

# Example values of x and y
model.Add(x == big_x_value)
model.Add(y == big_y_value)

# Compute the log. We set the precision to be low, as we deal with big values
log_x_var = log_of_x(model, x, float_precision_var=1, float_precision_image=log_float_precision)
log_y_var = log_of_x(model, y, float_precision_var=1, float_precision_image=log_float_precision)

# If we set the 
x_times_y = model.NewIntVar(1, 2*round(log(10**12+1000))*log_float_precision, "y")
model.Add(x_times_y == log_x_var + log_y_var)

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(
    f"Solver value of x*y is: {exp(solver.Value(x_times_y) / log_float_precision)} "
    + f"(Real value: {big_x_value * big_y_value})"
)
print(f"Relative error: {(exp(solver.Value(x_times_y) / log_float_precision) - (big_x_value * big_y_value) )/ (big_x_value * big_y_value)}")

Status = OPTIMAL
Solver value of x*y is: 9.999977681453931e+23 (Real value: 1000000000006999999999970)
Relative error: -2.23186160683225e-06


Despite the relative error being small, it's still huge in absolute value. It's an approximation. 

Note as well, that the support of the functions must be small enough, since we need to store the logarithm image for every value of the interval. 

A simpler way could be to compute `x//BIG_NUMBER` and `y//BIG_NUMBER`, then set `z == x//BIG_NUMBER * y//BIG_NUMBER `, and interpret it as `z*BIG_NUMBER**2`.

### Large product of many, many variables

You may not always get a large product of two single variables. But you might get a large products of numerous smaller variables. 

With the logarithm, we get for free product of multiple variables. That's because  and $\sum_k \log(x_k) = \log (\prod_k x_k )$

Once again, we are able to manipulate large quantities with an ok-level of approximation.

In [428]:
from random import randint

model = cp_model.CpModel()

float_precision_log = 100_000

n_variables = 100
many_variables = [model.NewIntVar(1, 100, f"{x}_{i}") for i in range(n_variables)]
many_log_variables = []

log_result = model.NewIntVar(
    1, 2 * float_precision_log * round(log(100)) * n_variables, "result"
)

# Set the var to be equal to random int between 1 and 10
real_product = 1
for i, var in enumerate(many_variables):
    random_value = randint(1, 100)
    if i < 10:
        print(f"Set {x}_{i}={random_value} ")
    model.Add(var == random_value)

    log_random_value = log_of_x(
        model, var, float_precision_var=1, float_precision_image=float_precision_log
    )
    many_log_variables.append(log_random_value)

    real_product *= random_value

print(f"... ({len(many_variables)} variables in total)")

model.Add(log_result == sum(many_log_variables))

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(
    f"Solution found by the solver is: {exp(solver.Value(log_result) / float_precision_log )}"
)
print(f"Real solution is: {real_product}")
print(
    f"Relative error: {(exp(solver.Value(log_result) / float_precision_log ) - real_product)/real_product}"
)


Set x_0=28 
Set x_1=74 
Set x_2=74 
Set x_3=38 
Set x_4=17 
Set x_5=66 
Set x_6=42 
Set x_7=71 
Set x_8=25 
Set x_9=86 
... (100 variables in total)
Status = OPTIMAL
Solution found by the solver is: 2.1290700900253401e+158
Real solution is: 212899174811596181578448858435179265963736896985091695902063357071093077381865583024923475729084353144425894505849610061214380949858418688000000000000000000000
Relative error: 3.679765759901138e-05


### Exponentiation

We know that $x^y = e^{y \log (x)}$. Hence, we can manipulate decimal powers in the solver, such as square root.

In [457]:
from math import sqrt

model = cp_model.CpModel()

float_precision_log = 10_000
value_of_x = 37

x = model.NewIntVar(1, 100, "x")
model.Add(x == value_of_x)

log_x = log_of_x(
    model, x, float_precision_var=1, float_precision_image=float_precision_log
)

sqrt_x = model.NewIntVar(1, 10 * float_precision_log * float_precision_log, "sqrt_x")
# we want sqrt_x == 0.5 * log_x
model.Add(float_precision_log * sqrt_x >= round(0.5 * float_precision_log) * log_x)

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f"Status = {solver.StatusName(status)}")
print(
    f"Solution found by the solver is: {exp(solver.Value(sqrt_x) / float_precision_log )}"
)
print(f"Real solution is: {sqrt(value_of_x)}")


Status = OPTIMAL
Solution found by the solver is: 6.083012194367712
Real solution is: 6.082762530298219
